In [1]:
import pycosat
import numpy as np

In [2]:
def sudoku_names(Dim):
    names = np.zeros([Dim,Dim,Dim], dtype = np.int)

    for i in range(Dim):
        for j in range(Dim):
            for k in range(Dim):
                names[i][j][k] = (i * Dim * Dim) + (j * Dim) + k +1
    return names

In [3]:
def encode_at_most_one(names):
    encode = []
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            arr = [-1*names[i], -1*names[j]]
            encode.insert(0, arr)
    return encode

In [4]:
def encode_at_least_one(names):
    return [names]

In [5]:
def encode_exactly_one(names):
    encode = []
    at_most_one = encode_at_most_one(names)
    at_least_one = encode_at_least_one(names)
    encode.extend(at_least_one)
    encode.extend(at_most_one)
    return encode

In [6]:
#add constrains
def encode_constraints(sudoku, Dim, names):
    encode = []
    for row in range(Dim):
        for column in range(Dim):
            if (sudoku[row,column] != 0):
                arr = [names[row,column, sudoku[row,column]-1].tolist()]
                encode.insert(0, arr)            
    return encode

In [7]:
def isSquare(number):
    if (np.sqrt(number)-int(np.sqrt(number))):
        return False
    else:
        return True

In [8]:
def get_block_positions(Dim):
    block_size = int(np.sqrt(Dim))
    block_positions = np.zeros([block_size], dtype = int)
    for pos in range(block_size):
        block_positions[pos] = block_size * pos
    
    return block_positions

In [9]:
def encode_sudoku(sudoku, Dim):
    names = sudoku_names(Dim)
    encode = []
    constraints = encode_constraints(sudoku, Dim, names)
    encode.extend(constraints)
    
    #for each cell, exactly one value
    for row in range(Dim):
        for column in range(Dim):
            cell_poss_values = names[row, column, :]
            cell_encode =  encode_exactly_one(cell_poss_values.tolist())
            encode.extend(cell_encode)
            
    #for each row, for each value, only one is true
    for row in range(Dim):
        for value in range(Dim):
            row_poss_values = names[row, :, value]
            row_encode = encode_exactly_one(row_poss_values.tolist())
            encode.extend(row_encode)
    
    #for each column, for each value, only one is true
    for column in range(Dim):
        for value in range(Dim):
            column_poss_values = names[:, column, value]
            column_encode = encode_exactly_one(column_poss_values.tolist())
            encode.extend(column_encode)
    
    #for each block, for each value, only one is true
    if isSquare(Dim):
        block_pos = get_block_positions(Dim)
        for row in block_pos:
            for column in block_pos:
                for value in range(Dim):
                    block_poss_values = names[row:row+block_pos[1], column:column+block_pos[1], value].flatten()
                    block_encode = encode_exactly_one(block_poss_values.tolist())
                    encode.extend(block_encode)
    
    return encode

In [ ]:
def decode_sudoku(solution, Dim):
    names = sudoku_names(Dim)
    sudoku = np.zeros([Dim, Dim], dtype = np.int)
    indexes = []
    for el in solution:
        if el > 0:
            index = np.where(names == el)
            for num in index:
                sudoku[index[0], index[1]] = index[2]+1
   
    print(sudoku)
   

In [ ]:
#s_test = np.array([[0,0], [1,0]])

#s_test = np.array([[3,4,1,0],
#                  [0,2,0,0],
#                  [0,0,2,0],
#                  [0,1,4,3]])

s_test = np.array([ [0,6,0,0,0,0,0,1,0],
                    [0,0,3,0,8,6,5,0,0],
                    [7,0,0,0,0,1,0,0,9],
                    [5,0,0,0,2,0,0,0,6],
                    [0,0,0,1,5,3,0,0,0],
                    [9,0,0,0,7,0,0,0,1],
                    [4,0,0,0,0,9,0,0,7],
                    [0,0,9,0,6,7,3,0,0],
                    [0,5,0,0,0,0,0,9,0]])
Dim = len(s_test[0])

encode = encode_sudoku(s_test, Dim)
solution = pycosat.solve(encode)
#print(solution)
#print(encode)
decode_sudoku(solution, Dim)
